In [ ]:
%matplotlib inline
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from chen_process import *

# load in data
data_path='../data_extraction/data/'

cohort = pd.read_table(data_path+'cohort.tsv')

cohort.ix[cohort.age > 90, 'age']=90

urine = pd.read_csv(data_path+'urine.csv')
urine = urine.loc[urine['value'] < 5000]

maps = pd.read_csv(data_path+'map.csv')
creatinine = pd.read_csv(data_path+'creatinine.csv')
admission_creatinine = pd.read_csv(data_path+'admission_creatinine.csv')


In [ ]:
# Interpolate the missing weights

cohort.set_index('icustay_id', inplace=True)
cohort = interpolateweights(cohort)

# Compute the urine blocks
aki_urine = calculateurineaki(urine, cohort)
cohort = pd.read_table(data_path+'cohort.tsv')

In [ ]:
# Get ethnicities 
eth = geteth(cohort)

In [ ]:
# clean the lactate
cohort = cohort.loc[cohort['max_lactate'] < 100]
cohort = cohort[pd.notnull(cohort['max_lactate'])]

In [ ]:
# Get map features 
mean_maps, min_maps, map_fracs = getmapfeatures(maps)

In [ ]:
# Process creatinine values and calculate aki 
creat_summary = analyzecreatinine(creatinine, admission_creatinine)

In [ ]:
# Get the logistic regression data
lr_data, aki_dataset, non_aki_dataset = getlrdata(cohort, eth, min_maps, creat_summary, aki_urine)

In [ ]:
# Summarize results

#AKI
print "------------------------------- AKI Patients -------------------------------"
print "============================================================================"
get_summary(aki_dataset)

#Non-AKI
print "----------------------------- Non-AKI Patients -----------------------------"
print "============================================================================"
get_summary(non_aki_dataset)

#Overall
print "------------------------------- All Patients -------------------------------"
print "============================================================================"
get_summary(lr_data)

In [ ]:
# Look at MAP split between AKI and Non-AKI subsets
bins = np.arange(20,80,1)

aki_dataset['min_map'].hist(alpha=0.5, bins = bins, label='AKI',normed=True)
non_aki_dataset['min_map'].hist(alpha=0.5, bins = bins, label='Non-AKI',normed=True)

plt.legend(loc='upper right')
plt.xlabel('Minimum Achieved MAP')
plt.ylabel('Fraction of Patients')
plt.show()

In [ ]:
# Organize data for logistic regression

train_cols = lr_data[['age','gender','eth','los','max_lactate','vaso_frac','min_map_bin']]
train_cols['gender'] = pd.Categorical(train_cols['gender'])
train_cols['eth'] = pd.Categorical(train_cols['eth'])
train_cols['min_map_bin'] = pd.Categorical(train_cols['min_map_bin'])

train_cols['gender'] = train_cols.gender.cat.codes
train_cols['eth'] = train_cols.eth.cat.codes
train_cols['min_map_bin'] = train_cols.min_map_bin.cat.codes

# Manually add the intercept
train_cols['intercept'] = 1.0

# Do logistic regression
logit = sm.Logit(lr_data['aki'], train_cols)
result = logit.fit()
print result.summary()

In [ ]:
# Get odds ratios
print np.exp(result.params)

In [ ]:
# The data split by the min_map_bin
mapgroups = []
akinums=np.zeros([7,3])
for binnum in range(0, 7):
    if binnum==6:
        binstring = '(90, 200]'
    else:
        binstring = '('+str(binnum+3)+'0, '+str(binnum+4)+'0]'
    mapgroup = lr_data.ix[lr_data.min_map_bin == binstring]
    mapgroups.append(mapgroup)
    
    akinums[binnum, 0]= mapgroup.loc[mapgroup['aki']==True].shape[0]
    akinums[binnum, 1] = mapgroup.shape[0] - akinums[binnum, 0]
    akinums[binnum, 2] = akinums[binnum,0]/mapgroup.shape[0]
    
# Bar chart of fraction of patients developing AKI within 72 hours for each minimum MAP bin
plt.bar(range(30, 91, 10), akinums[:,2], 10)